In [1]:
!pip install requests beautifulsoup4
!pip install html5lib


[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [2]:
import requests
from bs4 import BeautifulSoup
import re
from datetime import datetime

url = "https://nwis.waterdata.usgs.gov/nwis/qwdata?state_cd=ma&obs_date_range=1&qw_begin_date=08/01/2020&qw_end_date=2020-12-15&multiple_parameter_cds=53961%2C53962%2C54113%2C54116%2C54117%2C54220%2C54221%2C54222%2C54223%2C54224%2C54225%2C54226%2C54227%2C54228%2C54229%2C54230%2C54231%2C54232%2C54233%2C54234%2C54235%2C54236%2C54237%2C54238&param_cd_operator=AND&medium_cd=WS&group_key=NONE&sitefile_output_format=html_table&column_name=station_nm&inventory_output=0&rdb_inventory_output=file&TZoutput=0&pm_cd_compare=Greater%20than&radio_parm_cds=previous_parm_cds&format=html_table&qw_attributes=0&qw_sample_wide=wide&rdb_qw_attributes=0&date_format=YYYY-MM-DD&rdb_compression=file&list_of_search_criteria=state_cd%2Cobs_date_range%2Cmedium_cd%2Cmultiple_parameter_cds"

response = requests.get(url)
response.raise_for_status()

soup = BeautifulSoup(response.content, 'html.parser')

# Initialize empty lists to hold the data
names = []
pfhxs = []
pfoa = []
pfos = []
pfbs = []
pfba = [] 
pfds = []
pfda = []
pfdoa = []
pfhps = [] 
pfhpa = [] 
pfhxa = [] 
pfns = []
pfna = [] 
pfosa = []
pfpes = [] 
pfpea = []
pfta = [] 
pftrda = []
pfunda = [] 

# Extract water body names
water_bodies = soup.find_all('h3')
for water_body in water_bodies:
    name = water_body.get_text(strip=True)
    names.append(name)

In [3]:
import re

# Function to convert DMS to decimal degrees
def dms_to_decimal(dms_str):
    # Regular expression to match DMS format
    dms_parts = re.split('[°\'"]+', dms_str)
    dms_parts = [part for part in dms_parts if part]  # Remove any empty parts
    
    # Convert parts to decimal
    degrees = float(dms_parts[0])
    minutes = float(dms_parts[1]) if len(dms_parts) > 1 else 0
    seconds = float(dms_parts[2]) if len(dms_parts) > 2 else 0
    
    decimal_degrees = degrees + (minutes / 60) + (seconds / 3600)
    return decimal_degrees

# Your existing code
coordinates_divs = soup.find_all('div', align='left')

latitudes = []
longitudes = []

for coordinates_div in coordinates_divs:
    text = coordinates_div.get_text(strip=True).lower()  # Get and normalize the text
    
    # Split the text into words
    parts = text.split()

    # Initialize placeholders for the current div
    current_lat = None
    current_long = None

    # Iterate over the parts to find 'long' and 'lat' values
    for i in range(len(parts)):
        if parts[i] == 'longitude':
            current_long = parts[i + 1]
        elif parts[i] == 'latitude':
            current_lat = parts[i + 1]

    # Convert the DMS coordinates to decimal degrees
    if current_long is not None:
        longitudes.append(dms_to_decimal(current_long))
    if current_lat is not None:
        latitudes.append(dms_to_decimal(current_lat))

print(longitudes)
print(latitudes)


[71.74638888888889, 71.73722222222223, 71.81861111111111, 71.60972222222222, 71.63861111111112, 71.54638888888888, 71.45027777777779, 71.39055555555557, 71.39222222222223, 71.49527777777777, 71.49138888888889, 71.29861111111111, 71.29888888888888, 71.15313888888889, 71.39, 71.36277777777777, 70.97305555555556, 71.78765277777778, 71.78194444444445, 72.63305555555554, 72.62083333333332, 72.51444444444445, 72.8711111111111, 73.07288333333332, 73.24083333333333, 73.35466666666666, 73.15944444444445, 71.61638888888888, 71.68027777777779, 71.31388888888888, 71.28722222222223, 71.63861111111112, 71.61426666666667, 72.63630555555557, 72.31861111111111, 71.17661111111111, 70.98399722222223, 71.00980277777778, 71.68005833333333, 72.93435000000001, 71.75085833333333, 73.13815000000001, 71.52596944444444, 71.7273388888889, 71.15426666666667, 71.16041111111112, 70.92336111111112, 70.91362777777778, 71.50135833333333, 71.507, 72.49686111111112, 71.55976388888888, 71.56315555555555, 71.10255555555555

In [4]:
def clean_value(value):
    if isinstance(value, str):
        return value.replace('E', '').replace('<', '').replace('V', '').strip()
    return value

td_elements = soup.find_all('td')

def is_date_td(td):
    try:
        date_time_str = td.get_text(strip=True)
        datetime.strptime(date_time_str, '%Y-%m-%d %H:%M')
        return True
    except ValueError:
        return False

# List to store the indices of date elements
date_indices = []

# Find the indices of all date elements
for i, td in enumerate(td_elements):
    if is_date_td(td):
        date_indices.append(i)

for index in range(0, len(date_indices), 3): 
    start_index = date_indices[index]
    print(f'Starting at date: {td_elements[start_index].get_text(strip=True)}')
    
    if start_index + 7 < len(td_elements):
        hxs = clean_value(td_elements[start_index + 7].get_text(strip=True))
        print(f'pfhxs: {hxs}')
        pfhxs.append(hxs)
    
    if start_index + 8 < len(td_elements): 
        oa = clean_value(td_elements[start_index + 8].get_text(strip=True))
        print(f'pfoa: {oa}')
        pfoa.append(oa)
    
    if start_index + 9 < len(td_elements):
        os = clean_value(td_elements[start_index + 9].get_text(strip=True))
        print(f'pfos: {os}')
        pfos.append(os)
    
    if start_index + 13 < len(td_elements): 
        bs = clean_value(td_elements[start_index + 13].get_text(strip=True))
        print(f'pfbs: {bs}')
        pfbs.append(bs)
    
    if start_index + 14 < len(td_elements): 
        ba = clean_value(td_elements[start_index + 14].get_text(strip=True))
        print(f'pfba: {ba}')
        pfba.append(ba)
    
    if start_index + 15 < len(td_elements): 
        ds = clean_value(td_elements[start_index + 15].get_text(strip=True))
        print(f'pfds: {ds}')
        pfds.append(ds)
    
    if start_index + 16 < len(td_elements): 
        da = clean_value(td_elements[start_index + 16].get_text(strip=True))
        print(f'pfda: {da}')
        pfda.append(da)
    
    if start_index + 17 < len(td_elements): 
        doa = clean_value(td_elements[start_index + 17].get_text(strip=True))
        print(f'pfdoa: {doa}')
        pfdoa.append(doa)
    
    if start_index + 18 < len(td_elements): 
        hps = clean_value(td_elements[start_index + 18].get_text(strip=True))
        print(f'pfhps: {hps}')
        pfhps.append(hps)
    
    if start_index + 19 < len(td_elements): 
        hpa = clean_value(td_elements[start_index + 19].get_text(strip=True))
        print(f'pfhpa: {hpa}')
        pfhpa.append(hpa)
    
    if start_index + 20 < len(td_elements): 
        hxa = clean_value(td_elements[start_index + 20].get_text(strip=True))
        print(f'pfhxa: {hxa}')
        pfhxa.append(hxa)
    
    if start_index + 21 < len(td_elements): 
        ns = clean_value(td_elements[start_index + 21].get_text(strip=True))
        print(f'pfns: {ns}')
        pfns.append(ns)
    
    if start_index + 22 < len(td_elements): 
        na = clean_value(td_elements[start_index + 22].get_text(strip=True))
        print(f'pfna: {na}')
        pfna.append(na)
    
    if start_index + 23 < len(td_elements): 
        osa = clean_value(td_elements[start_index + 23].get_text(strip=True))
        print(f'pfosa: {osa}')
        pfosa.append(osa)
    
    if start_index + 24 < len(td_elements): 
        pes = clean_value(td_elements[start_index + 24].get_text(strip=True))
        print(f'pfpes: {pes}')
        pfpes.append(pes)
    
    if start_index + 25 < len(td_elements): 
        pea = clean_value(td_elements[start_index + 25].get_text(strip=True))
        print(f'pfpea: {pea}')
        pfpea.append(pea)
    
    if start_index + 26 < len(td_elements): 
        ta = clean_value(td_elements[start_index + 26].get_text(strip=True))
        print(f'pfta: {ta}')
        pfta.append(ta)
    
    if start_index + 27 < len(td_elements): 
        trda = clean_value(td_elements[start_index + 27].get_text(strip=True))
        print(f'pftrda: {trda}')
        pftrda.append(trda)
    
    if start_index + 28 < len(td_elements): 
        unda = clean_value(td_elements[start_index + 28].get_text(strip=True))
        print(f'pfunda: {unda}')
        pfunda.append(unda)


Starting at date: 2020-08-28 12:30
pfhxs: 4.44
pfoa: 26.9
pfos: 9.40
pfbs: 59.7
pfba: 31.0
pfds: 1.81
pfda: 3.68
pfdoa: 1.81
pfhps: 1.81
pfhpa: 13.8
pfhxa: 55.7
pfns: 1.81
pfna: 2.65
pfosa: 1.81
pfpes: 0.637
pfpea: 36.7
pfta: 1.81
pftrda: 1.81
pfunda: 1.81
Starting at date: 2020-08-28 14:00
pfhxs: 4.15
pfoa: 26.5
pfos: 10.1
pfbs: 49.6
pfba: 28.4
pfds: 1.78
pfda: 4.03
pfdoa: 1.78
pfhps: 1.78
pfhpa: 13.5
pfhxa: 49.1
pfns: 1.78
pfna: 2.57
pfosa: 1.78
pfpes: 0.528
pfpea: 33.7
pfta: 1.78
pftrda: 1.78
pfunda: 1.78
Starting at date: 2020-08-24 14:00
pfhxs: 1.92
pfoa: 3.72
pfos: 1.46
pfbs: 1.12
pfba: 1.58
pfds: 1.92
pfda: 1.92
pfdoa: 1.92
pfhps: 1.92
pfhpa: 2.24
pfhxa: 5.08
pfns: 1.92
pfna: 0.548
pfosa: 1.92
pfpes: 1.92
pfpea: 4.18
pfta: 1.92
pftrda: 1.92
pfunda: 1.92
Starting at date: 2020-08-20 17:30
pfhxs: 6.93
pfoa: 21.3
pfos: 12.6
pfbs: 20.5
pfba: 13.5
pfds: 1.78
pfda: 1.28
pfdoa: 1.78
pfhps: 1.78
pfhpa: 10.8
pfhxa: 28.2
pfns: 1.78
pfna: 1.72
pfosa: 1.78
pfpes: 0.626
pfpea: 27.8
pfta: 1.7

In [5]:
water_data = []
for i in range(len(names)):
    water_body_data = {
        "name": names[i],
        "longitude": longitudes[i] if i < len(longitudes) else None,
        "latitude": latitudes[i] if i < len(latitudes) else None, 
        "pfhxs": clean_value(pfhxs[i]) if i < len(pfhxs) else None,
        "pfoa": clean_value(pfoa[i]) if i < len(pfoa) else None,
        "pfos": clean_value(pfos[i]) if i < len(pfos) else None,
        "pfbs": clean_value(pfbs[i]) if i < len(pfbs) else None,
        "pfba": clean_value(pfba[i]) if i < len(pfba) else None,
        "pfds": clean_value(pfds[i]) if i < len(pfds) else None,
        "pfda": clean_value(pfda[i]) if i < len(pfda) else None,
        "pfdoa": clean_value(pfdoa[i]) if i < len(pfdoa) else None,
        "pfhps": clean_value(pfhps[i]) if i < len(pfhps) else None,
        "pfhpa": clean_value(pfhpa[i]) if i < len(pfhpa) else None,
        "pfhxa": clean_value(pfhxa[i]) if i < len(pfhxa) else None,
        "pfns": clean_value(pfns[i]) if i < len(pfns) else None,
        "pfna": clean_value(pfna[i]) if i < len(pfna) else None,
        "pfosa": clean_value(pfosa[i]) if i < len(pfosa) else None,
        "pfpes": clean_value(pfpes[i]) if i < len(pfpes) else None,
        "pfpea": clean_value(pfpea[i]) if i < len(pfpea) else None,
        "pfta": clean_value(pfta[i]) if i < len(pfta) else None,
        "pftrda": clean_value(pftrda[i]) if i < len(pftrda) else None,
        "pfunda": clean_value(pfunda[i]) if i < len(pfunda) else None 
    }
    water_data.append(water_body_data)

# Print the organized data
for data in water_data:
    print(f"Water Body Name: {data['name']}")
    print(f"Longitude: {data['longitude']}")
    print(f"Latitude: {data['latitude']}")
    print(f"PFOA: {data['pfoa']}")
    print(f"PFOS: {data['pfos']}")
    print(f"PFhXS: {data['pfhxs']}")
    print(" ")


Water Body Name: USGS 01094450 NORTH NASHUA RIVER AT NORTH LEOMINSTER, MA
Longitude: 71.74638888888889
Latitude: 42.54222222222222
PFOA: 26.9
PFOS: 9.40
PFhXS: 4.44
 
Water Body Name: USGS 01094455 NORTH NASHUA RIVER AT LEOMINSTER, MA
Longitude: 71.73722222222223
Latitude: 42.52583333333333
PFOA: 26.5
PFOS: 10.1
PFhXS: 4.15
 
Water Body Name: USGS 01094950 STILLWATER RIVER AT WEST STERLING, MA
Longitude: 71.81861111111111
Latitude: 42.446666666666665
PFOA: 3.72
PFOS: 1.46
PFhXS: 1.92
 
Water Body Name: USGS 01095920 NASHUA RIVER NEAR AYER, MA
Longitude: 71.60972222222222
Latitude: 42.57833333333333
PFOA: 21.3
PFOS: 12.6
PFhXS: 6.93
 
Water Body Name: USGS 01096603 ASSABET RIVER NEAR WESTBOROUGH, MA
Longitude: 71.63861111111112
Latitude: 42.279444444444444
PFOA: 5.13
PFOS: 9.39
PFhXS: 2.66
 
Water Body Name: USGS 01096870 ASSABET RIVER AT COX ST NEAR HUDSON, MA
Longitude: 71.54638888888888
Latitude: 42.39972222222222
PFOA: 9.49
PFOS: 7.62
PFhXS: 2.86
 
Water Body Name: USGS 01097000 ASS

In [6]:
print(len(pfhxs))
del pfhxs[64:576]

print(len(pfoa)) 
del pfoa[64:576]

print(len(pfos))
del pfos[64:576]

print(len(pfbs))
del pfbs[64:768]

print(len(pfba))
del pfba[64:576]

print(len(pfds))
del pfds[64:576]

print(len(pfda))
del pfda[64:576]

print(len(pfdoa))
del pfdoa[64:576]


print(len(pfhps))
del pfhps[64:576]
print(len(pfhpa))
del pfhpa[64:576]
print(len(pfhxa))
del pfhxa[64:512]
print(len(pfns))
del pfns[64:576]
print(len(pfna))
del pfna[64:512]
print(len(pfosa))
del pfosa[64:512]
print(len(pfpes)) 
del pfpes[64:512]
print(len(pfpea))
del pfpea[64:512]
print(len(pfta))

del pfta[64:512]
print(len(pftrda))
del pftrda[64:512]

del pfunda[64:512]
print(len(pfunda))
print(len(longitudes))
print(len(latitudes))

64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64
64


In [7]:
import pandas as pd

# Example DataFrame creation
df = pd.DataFrame({
    'Name': names, 
    'Longitude': longitudes,
    'Latitude': latitudes, 
    'PFHxS': pfhxs,
    'PFOA': pfoa, 
    'PFOS': pfos,
    'PFBS': pfbs, 
    'PFBA': pfba, 
    'PFDS': pfds, 
    'PFDA': pfda,
    'PFDoA': pfdoa,
    'PFHPS': pfhps,
    'PFHPA': pfhpa,
    'PFHxA': pfhxa,
    'PFNS': pfns,
    'PFNA': pfna,
    'PFOSA': pfosa,
    'PFPES': pfpes, 
    'PFPEA': pfpea, 
    'PFTA': pfta, 
    'PFTRDA': pftrda, 
    'PFuNDA': pfunda, 
})

df = df.applymap(clean_value)

selected_columns = df.drop(columns=['Name','Longitude', 'Latitude']).columns
df[selected_columns] = df[selected_columns].apply(pd.to_numeric, errors='coerce')

df['Total_PFAS'] = df[selected_columns].sum(axis=1)

df.head()


,Name,Longitude,Latitude,PFHxS,PFOA,PFOS,PFBS,PFBA,PFDS,PFDA,...,PFHxA,PFNS,PFNA,PFOSA,PFPES,PFPEA,PFTA,PFTRDA,PFuNDA,Total_PFAS
0,USGS 01094450 NORTH NASHUA RIR AT NORTH LOMINS...,71.746389,42.542222,4.44,26.90,9.40,59.70,31.00,1.81,3.680,...,55.70,1.81,2.650,1.81,0.637,36.70,1.81,1.81,1.81,259.087
1,"USGS 01094455 NORTH NASHUA RIR AT LOMINSTR, MA",71.737222,42.525833,4.15,26.50,10.10,49.60,28.40,1.78,4.030,...,49.10,1.78,2.570,1.78,0.528,33.70,1.78,1.78,1.78,236.418
2,"USGS 01094950 STILLWATR RIR AT WST STRLING, MA",71.818611,42.446667,1.92,3.72,1.46,1.12,1.58,1.92,1.920,...,5.08,1.92,0.548,1.92,1.920,4.18,1.92,1.92,1.92,41.048
3,"USGS 01095920 NASHUA RIR NAR AYR, MA",71.609722,42.578333,6.93,21.30,12.60,20.50,13.50,1.78,1.280,...,28.20,1.78,1.720,1.78,0.626,27.80,1.78,1.78,1.78,159.496
4,"USGS 01096603 ASSABT RIR NAR WSTBOROUGH, MA",71.638611,42.279444,2.66,5.13,9.39,1.90,6.21,1.86,0.659,...,4.59,1.86,10.100,1.86,0.369,5.06,1.86,1.86,1.84,63.658


In [8]:
df.to_csv('water_bodies.csv')